In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: ContinuousCallbackPMS - ContinuousPMS Sonecule using a Callback function for flexible mapping

This notebook introduces and demonstrates usage of the ContinuousCallbackPMS sonecule.
* The sonecule spawns a synth to be modulated by a data-mapped parameters
* The synths can be defined as wished with parameters as needed
* The default parameter offers the following parameters:
  * amplitude
  * frequency
  * sharpness
  * spatial panning
* Different from the ContinuousPMS, where the mapping is specified by a mapping specification dictionary that is parsed column by column, this sonecule calls a callback function for each row, providing the row vector as an argument. 
* this allows highly flexible mappings and even things that are impossible with the non-callback Sonecule, such as playing a sound that conveys which channel has the highest value.
* Definition of a callback function is a daunting task for non-programmers. Therefore the sonecule comes with a create_callback_template method, that delivers a function that can be copied and pasted in a jupyter notebook cell (or your IDE), as starting point for own sonification designs.


## Data Preparation

In [ ]:
# headers and imports for the demo
import sonecules as sn
import sc3nb as scn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt
import time

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

In [ ]:
%run ../examples/prepare-data.ipynb

In [ ]:
df = dataframes["building"]
df.plot(lw=0.2);

## Usage / Demo Snippets

Let's create a ContinuousCallbackPMS using a callback function for mapping.

First we need to import the sonecule

In [ ]:
from sonecules.scoreson import ContinuousCallbackPMS

The following cell shows all steps usually used to sonify the data, i.e., 
- creation of the sonecule
- cleaning the time line
- defining the callback function (here named callback_fn, but any name is fine)
- creating the sonification using the sonecules schedule function
- starting your sonification using the sonecule start function

For the example we start with one week =7*24 hourly measurments of the building data set

In [ ]:
# create the sonecule
scb = ContinuousCallbackPMS(df.iloc[:7*24, 6:])  # one week, omit the first 6 columns

# reset the timeline
ctx.timeline.reset()

# this is just a shortcut to the mapcol function which maps the columns data 
# mapcol(r, feature, column_mins, column_maxs, target_min, target_max)
mapcol = scb.mapcol

# define the callback function as needed/wanted
def callback_fn(r, cmi, cma, pp):
    pp['freq']      = pam.midicps(mapcol(r, 'temperature', cmi, cma, 48, 72))
    pp['amp']       = pam.dbamp(mapcol(r, 'humidity', cmi, cma, -20, 0)) 
    pp['pan']       = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    pp['numharm']   = mapcol(r, 'solar_radiation', cmi, cma, 1, 12)
    pp['vibfreq']   = scn.linlin(r['hc_wb_hot_water'], -0.5, 0.5, 3, 8)
    pp['vibintrel'] = 0
    return pp # it returns the mapping dictionary

# sonify the data using the above callback function
scb.schedule(at=0, duration=6, callback_fn=callback_fn)

# start the playback 
scb.start(rate=1)

- play with the definition of the callback_fn and execute again to explore the data
- control the duration and data slice.
- Once you want to keep the callback function fixed there is no need to reexecute
- Here some other data sonified with the same callback function

In [ ]:
scb = ContinuousCallbackPMS(df.iloc[30*24:60*24, 7:])  # the second month of data in 10 seconds
ctx.timeline.reset()
scb.schedule(at=0, duration=10, callback_fn=callback_fn).start()

In [ ]:
scb = ContinuousCallbackPMS(df.iloc[::, 7:])  # all data, every 2nd row (>6m) in 15 seconds
ctx.timeline.reset()
if ctx.playback.running: 
    ctx.playback.stop()
scb.schedule(at=0, duration=2, callback_fn=callback_fn).start()

- As this is all executed in real-time, you will probably experience limits of your system.
- Sonecules will issue late warnings if there are processing-based delays
- in such cases, a non-real-time rendering is always an option to compute a guaranteed correct sonification. This will be demoed elsewhere. The plan is to integrate this as an option into all sonecules.


In [ ]:
ctx.stop()  # execute this in case an error occurs and synths keep on playing

Now let the sonecule propose a mapping as starting point for own experiments.

- Calling `create_callback_template()` composes 
- and then prints the python code string.
- This output can be copied into a notebook cell and adapted as needed.
- On execution it defines the callback function cbfn().
- The sonification can be rendered as you go until satisfactory.

In [ ]:
scb = ContinuousCallbackPMS(df.iloc[:7*24, 8:])  # first week but no weekday features
fnstr = scb.create_callback_template(auto_assign=True)

The printout shows python code 
- that you can simply copy and paste into a new code cell
- modify the cell as you see fit, e.g. remove rows, or set the right hand side to a constant as preferred
- once satisfied, you can execute the function.
- Note that the 


In [ ]:
def cbfn(r, cmi, cma, pp):
    # columns are:'hour' 'am_pm' 'temperature' 'humidity' 
    # 'solar_radiation' 'wind_speed' 'hc_wb_electrical' 'hc_wb_cold_water' 
    # 'hc_wb_hot_water' 
    pp['freq']     	 = mapcol(r, 'hour', cmi, cma, 300.00, 600.00)
    pp['amp']      	 = mapcol(r, 'am_pm', cmi, cma, 0.08, 0.15)
    pp['vibfreq']  	 = mapcol(r, 'temperature', cmi, cma, 0.00, 0.00)
    pp['vibintrel']	 = mapcol(r, 'humidity', cmi, cma, 0.00, 0.00)
    pp['numharm']  	 = mapcol(r, 'solar_radiation', cmi, cma, 0.00, 10.00)
    pp['pulserate']	 = mapcol(r, 'wind_speed', cmi, cma, 0.00, 0.00)
    pp['pint']     	 = mapcol(r, 'hc_wb_electrical', cmi, cma, 0.00, 0.00)
    pp['pwid']     	 = mapcol(r, 'hc_wb_cold_water', cmi, cma, 0.75, 1.50)
    pp['pan']      	 = mapcol(r, 'hour', cmi, cma, -1.00, 1.00)
    return pp
# create sonification e.g. by using
sn.gcc().timeline.reset()
scb.schedule(at=0, duration=5, callback_fn=cbfn).start(rate=1)

And finally here is a hand-crafted mapping, from modifying the code

In [ ]:
scb = ContinuousCallbackPMS(df.iloc[14*24:18*24, 8:]) 

def cbfn(r, cmi, cma, pp):
    # columns are:
    # 'hour' 'am_pm' 'temperature' 'humidity' 
    # 'solar_radiation' 'wind_speed' 'hc_wb_electrical' 'hc_wb_cold_water' 
    # 'hc_wb_hot_water' 
    # print(r, pp)
    pp['freq']	     = mapcol(r, 'solar_radiation', cmi, cma, 100, 800)
    pp['amp']	     = mapcol(r, 'humidity', cmi, cma, 0, 1)
    pp['numharm']	 = mapcol(r, 'am_pm', cmi, cma, 1, 4)
    pp['vibintrel']  = 0
    pp['pan']	     = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    return pp

ctx.timeline.reset()
scb.schedule(at=1, duration=8, callback_fn=cbfn).start(rate=1)

In [ ]:
ctx.backend.stop(ctx)